In [ ]:
from hyperch4 import Hyper
from hyperch4.folium_map import Map
from glob import glob
import xarray as xr
import numpy as np
import folium
import matplotlib.pyplot as plt

In [ ]:
data_dir = '/Users/xinz/Documents/github/HyperCH4/test_data/'

## PRISMA

In [ ]:
filename = glob(data_dir+'PRS_L1_STD_OFFL_20211021182310_20211021182315_0001.he5')
hyp_prisma = Hyper(filename, reader='hyc_l1') # "hsi_l1b", "emit_l1b", "hyc_l1"

hyp_prisma.load()

In [ ]:
hyp_prisma.retrieve(wvl_intervals=[1300, 2500])
ch4_swir = hyp_prisma.scene['ch4']
hyp_prisma.retrieve(wvl_intervals=[2100, 2450])
ch4 = hyp_prisma.scene['ch4']

# calculate ch4_comb
diff = ch4_swir - ch4
scale = ch4.std()/ch4_swir.std()
# scale if ch4_swir < ch4
hyp_prisma.scene['ch4_comb'] = ch4.where(diff > 0, ch4_swir*scale).rename('ch4_comb')

In [ ]:
# remove the bands dim for ch4
hyp_prisma.scene['ch4'] = hyp_prisma.scene['ch4'].isel(bands=0)
hyp_prisma.scene['ch4_comb'] = hyp_prisma.scene['ch4_comb'].isel(bands=0)

In [ ]:
hyp_prisma.scene['ch4_denoise'] = hyp_prisma.denoise(varname='ch4')
hyp_prisma.scene['ch4_comb_denoise'] = hyp_prisma.denoise(varname='ch4_comb')

In [ ]:
vnames = ['u10', 'v10', 'rgb', 'ch4', 'ch4_comb', 'ch4_denoise', 'ch4_comb_denoise']

# export to Dataset
ds_prisma = hyp_prisma.scene.to_xarray(datasets=vnames)

# export to netcdf file
# hyp_prisma.scene.save_datasets(datasets=vnames, filename='test.nc', writer='cf')

In [ ]:
m_prisma = Map(ds_prisma, ['rgb', 'ch4', 'ch4_comb', 'ch4_comb_denoise'])
m_prisma.initialize()
m_prisma.plot(show_layers=[False, False, False, True],
       opacities=[0.9, 0.7, 0.7, 0.7])

# show the map in notebook
layer_control = folium.LayerControl(collapsed=False, position='topleft', draggable=True)
m_prisma.map.add_child(layer_control)

# export to html file
# m_prisma.export()

## EnMAP

In [ ]:
filename = glob(data_dir+'ENMAP01-____L1B-DT0000015090_20230424T060259Z_001_V010201_20230426T035016Z.ZIP')
hyp_enmap = Hyper(filename, reader='hsi_l1b') # "hsi_l1b", "emit_l1b", "hyc_l1"

hyp_enmap.load()

In [ ]:
hyp_enmap.retrieve(wvl_intervals=[1300, 2500])
ch4_swir = hyp_enmap.scene['ch4']
hyp_enmap.retrieve(wvl_intervals=[2100, 2450])
ch4 = hyp_enmap.scene['ch4']

# calculate ch4_comb
diff = ch4_swir - ch4
scale = ch4.std()/ch4_swir.std()
# scale if ch4_swir < ch4
hyp_enmap.scene['ch4_comb'] = ch4.where(diff > 0, ch4_swir*scale).rename('ch4_comb')

In [ ]:
hyp_enmap.scene['ch4_denoise'] = hyp_enmap.denoise(varname='ch4')
hyp_enmap.scene['ch4_comb_denoise'] = hyp_enmap.denoise(varname='ch4_comb')

In [ ]:
def ortho_enmap(hyp):
    rgb_corr = hyp.terrain_corr(varname='rgb', rpcs=hyp.scene['rpc_coef_vnir'].sel(bands_vnir=650, method='nearest').item())
    ch4_corr = hyp.terrain_corr(varname='ch4', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())
    ch4_comb_corr = hyp.terrain_corr(varname='ch4_comb', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())
    ch4_denoise_corr = hyp.terrain_corr(varname='ch4_denoise', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())
    ch4_comb_denoise_corr = hyp.terrain_corr(varname='ch4_comb_denoise', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())

    rgb = rgb_corr.interp_like(ch4_corr)
    rgb.attrs['area'] = ch4_corr.attrs['area']

    ch4 = ch4_corr.isel(bands=0)
    ch4_comb = ch4_comb_corr.isel(bands=0)
    ch4_denoise = ch4_denoise_corr.isel(bands=0)
    ch4_comb_denoise = ch4_comb_denoise_corr.isel(bands=0)

    hyp.scene['rgb'] = rgb
    hyp.scene['ch4'] = ch4
    hyp.scene['ch4_comb'] = ch4_comb
    hyp.scene['ch4_denoise'] = ch4_denoise
    hyp.scene['ch4_comb_denoise'] = ch4_comb_denoise

In [ ]:
ortho_enmap(hyp_enmap)

In [ ]:
vnames = ['u10', 'v10', 'rgb', 'ch4', 'ch4_comb', 'ch4_denoise', 'ch4_comb_denoise']

# export to Dataset
ds_enmap = hyp_enmap.scene.to_xarray(datasets=vnames)

# export to netcdf file
# hyp_enmap.scene.save_datasets(datasets=vnames, filename='test.nc', writer='cf')

In [ ]:
m_enmap = Map(ds_enmap, ['rgb', 'ch4', 'ch4_comb','ch4_comb_denoise'])
m_enmap.initialize()
m_enmap.plot(show_layers=[False, False, False, True],
       opacities=[0.9, 0.7, 0.7, 0.7])

# show the map in notebook
layer_control = folium.LayerControl(collapsed=False, position='topleft', draggable=True)
m_enmap.map.add_child(layer_control)

# export to html file
# m_enmap.export()

## EMIT

In [ ]:
filename = glob(data_dir+'EMIT_L1B_*_001_20230427T064416_2311704_004.nc')
hyp_emit = Hyper(filename, reader='emit_l1b') # "hsi_l1b", "emit_l1b", "hyc_l1"

hyp_emit.load()

In [ ]:
hyp_emit.retrieve(wvl_intervals=[1300, 2500])
ch4_swir = hyp_emit.scene['ch4']
hyp_emit.retrieve(wvl_intervals=[2100, 2450])
ch4 = hyp_emit.scene['ch4']

# calculate ch4_comb
diff = ch4_swir - ch4
scale = ch4.std()/ch4_swir.std()
# scale if ch4_swir < ch4
hyp_emit.scene['ch4_comb'] = ch4.where(diff > 0, ch4_swir*scale).rename('ch4_comb')

In [ ]:
hyp_emit.scene['ch4_denoise'] = hyp_emit.denoise(varname='ch4')
hyp_emit.scene['ch4_comb_denoise'] = hyp_emit.denoise(varname='ch4_comb')

In [ ]:
def ortho_emit(hyp):
    # orthorectification
    rgb_corr = hyp.terrain_corr(varname='rgb')
    ch4_corr = hyp.terrain_corr(varname='ch4')
    ch4_comb_corr = hyp.terrain_corr(varname='ch4_comb')
    ch4_denoise_corr = hyp.terrain_corr(varname='ch4_denoise')
    ch4_comb_denoise_corr = hyp.terrain_corr(varname='ch4_comb_denoise')

    # save into Scene
    hyp.scene['rgb'] = rgb_corr
    hyp.scene['ch4'] = ch4_corr.isel(bands=0)
    hyp.scene['ch4_comb'] = ch4_comb_corr.isel(bands=0)
    hyp.scene['ch4_denoise'] = ch4_denoise_corr.isel(bands=0)
    hyp.scene['ch4_comb_denoise'] = ch4_comb_denoise_corr.isel(bands=0)

In [ ]:
ortho_emit(hyp_emit)

In [ ]:
vnames = ['u10', 'v10', 'rgb', 'ch4', 'ch4_comb', 'ch4_denoise', 'ch4_comb_denoise']

# export to Dataset
ds_emit = hyp_emit.scene.to_xarray(datasets=vnames)

# export to netcdf file
# hyp_emit.scene.save_datasets(datasets=vnames, filename='test.nc', writer='cf')

In [ ]:
m_emit = Map(ds_emit, ['rgb', 'ch4', 'ch4_comb','ch4_comb_denoise'])
m_emit.initialize()
m_emit.plot(show_layers=[False, False, False, True],
       opacities=[0.9, 0.7, 0.7, 0.7])

# show the map in notebook
layer_control = folium.LayerControl(collapsed=False, position='topleft', draggable=True)
m_emit.map.add_child(layer_control)

# export to html file
# m_emit.export()